## 我们的任务 ##

垃圾邮件检测是机器学习在现今互联网领域的主要应用之一。几乎所有大型电子邮箱服务提供商都内置了垃圾邮件检测系统，能够自动将此类邮件分类为“垃圾邮件”。 

在此项目中，我们将使用朴素贝叶斯算法创建一个模型，该模型会通过我们对模型的训练将信息[数据集](https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection)分类为垃圾信息或非垃圾信息。对垃圾文本信息进行大致了解十分重要。通常它们都包含“免费”、“赢取”、“获奖者”、“现金”、“奖品”等字眼，因为这些它们专门用来吸引你的注意力，诱惑你打开信息。此外，垃圾信息的文字一般都使用大写形式和大量感叹号。收信人能轻易辨认垃圾信息，而我们的目标是训练模型帮助我们识别垃圾信息！

能够识别垃圾信息是一种二元分类问题，因为此处信息只有“垃圾信息”或“非垃圾信息”这两种分类。此外，这是一种监督式学习问题，因为我们会向模型中提供带标签数据集，模型能够从中学习规律并在日后做出预测。


### 第 1.1 步：了解我们的数据集 ### 


我们将使用来自 UCI 机器学习资源库中的[数据集](https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection)，该资源库有大量供实验性研究的精彩数据集。[这是](https://archive.ics.uci.edu/ml/machine-learning-databases/00228/)直接数据链接。


 ** 下面是该数据的预览：** 

<img src="images/dqnb.png" height="1242" width="1242">

数据集中的列目前没有命名，可以看出有 2 列。

第一列有两个值：“ham”，表示信息不是垃圾信息，以及“spam”，表示信息是垃圾信息。

第二列是被分类的信息的文本内容。

In [1]:
import pandas as pd
# Dataset from - https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection
df = pd.read_table("smsspamcollection/SMSSpamCollection", 
                   sep="\t", 
                   header=None, 
                   names=['label', 'sms_message'])

# Output printing out first 5 columns
df.head()

,label,sms_message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


### 第 1.2 步：数据预处理 ###

我们已经大概了解数据集的结构，现在将标签转换为二元变量，0 表示“ham”（即非垃圾信息），1表示“spam”，这样比较方便计算。

In [2]:
df['label'] = df.label.map({'ham': 0, 'spam': 1})
print (df.shape)

(5572, 2)


### 第 2 步：从头实现 Bag of Words ###

在深入了解帮助我们处理繁重工作的 scikit-learn 的 Bag of Words(BoW) 库之前，首先我们自己实现该步骤，以便了解该库的背后原理。 

** 第 1 步：将所有字符串转换成小写形式。**

假设有一个文档集合：

In [5]:
documents = ['Hello, how are you!',
             'Win money, win from home.',
             'Call me now.',
             'Hello, Call hello you tomorrow?']

lower_case_documents = [s.lower() for s in documents]
print(lower_case_documents)

['hello, how are you!', 'win money, win from home.', 'call me now.', 'hello, call hello you tomorrow?']


** 第 2 步：删除所有标点符号 **

In [6]:
sans_punctuation_documents = []
import re

for i in lower_case_documents:
    sans_punctuation_documents.append(re.sub(r'[^A-Za-z0-9]', ' ', i).strip())
    
print(sans_punctuation_documents)

['hello  how are you', 'win money  win from home', 'call me now', 'hello  call hello you tomorrow']


** 第 3 步：Tokenization **

In [7]:
preprocessed_documents = []
for i in sans_punctuation_documents:
    preprocessed_documents.append(i.split())
print(preprocessed_documents)

[['hello', 'how', 'are', 'you'], ['win', 'money', 'win', 'from', 'home'], ['call', 'me', 'now'], ['hello', 'call', 'hello', 'you', 'tomorrow']]


** 第 4 步：计算频率 ** 

In [37]:
frequency_list = []
import pprint
from collections import Counter

for i in preprocessed_documents:
    frequency_list.append(Counter(i))
    
pprint.pprint(frequency_list)

[Counter({'hello': 1, 'how': 1, 'are': 1, 'you': 1}),
 Counter({'win': 2, 'money': 1, 'from': 1, 'home': 1}),
 Counter({'call': 1, 'me': 1, 'now': 1}),
 Counter({'hello': 2, 'call': 1, 'you': 1, 'tomorrow': 1})]


### 第 2.3 步：在 scikit-learn 中实现 Bag of Words ###

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
count_vector = CountVectorizer()

** 使用 CountVectorizer() 预处理数据 **

In [38]:
print(count_vector)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)


In [12]:
count_vector.fit(documents)
count_vector.get_feature_names()

['are',
 'call',
 'from',
 'hello',
 'home',
 'how',
 'me',
 'money',
 'now',
 'tomorrow',
 'win',
 'you']

In [13]:
doc_array = count_vector.transform(documents).toarray()
doc_array

array([[1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 2, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0],
       [0, 1, 0, 2, 0, 0, 0, 0, 0, 1, 0, 1]], dtype=int64)

In [14]:
frequency_matrix = pd.DataFrame(doc_array, columns=count_vector.get_feature_names())
frequency_matrix

,are,call,from,hello,home,how,me,money,now,tomorrow,win,you
0,1,0,0,1,0,1,0,0,0,0,0,1
1,0,0,1,0,1,0,0,1,0,0,2,0
2,0,1,0,0,0,0,1,0,1,0,0,0
3,0,1,0,2,0,0,0,0,0,1,0,1


### 第 3.1 步：训练集和测试集 ###

In [15]:
# split into training and testing sets
# USE from sklearn.model_selection import train_test_split to avoid seeing deprecation warning.
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['sms_message'], 
                                                    df['label'], 
                                                    random_state=1)

print('Number of rows in the total set: {}'.format(df.shape[0]))
print('Number of rows in the training set: {}'.format(X_train.shape[0]))
print('Number of rows in the test set: {}'.format(X_test.shape[0]))

Number of rows in the total set: 5572
Number of rows in the training set: 4179
Number of rows in the test set: 1393


D:\Users\YangY\Anaconda2\envs\python3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### 第 3.2 步：对数据集应用 Bag of Words 流程。 ###

In [17]:
# Instantiate the CountVectorizer method
count_vector = CountVectorizer()

# Fit the training data and then return the matrix
training_data = count_vector.fit_transform(X_train)

# Transform testing data and return the matrix. Note we are not fitting the testing data into the CountVectorizer()
testing_data = count_vector.transform(X_test)

### 第 4.1 步：从头实现贝叶斯定理 ###

我们的数据集已经是我们希望的格式，现在可以进行任务的下一步了，即研究用来做出预测并将信息分类为垃圾信息或非垃圾信息的算法。记得在该项目的开头，我们简要介绍了贝叶斯定理，现在我们将深入讲解该定理。通俗地说，贝叶斯定理根据与相关事件有关的其他事件的概率计算该事件的发生概率。它由先验概率（我们知道的概率或提供给我们的概率）和后验概率（我们希望用先验部分计算的概率）组成。

我们用一个简单的示例从头实现贝叶斯定理。假设我们要根据某人接受糖尿病检测后获得阳性结果计算此人有糖尿病的概率。
在医学领域，此类概率非常重要，因为它们涉及的是生死情况。 

我们假设：

`P(D)` 是某人患有糖尿病的概率。值为  `0.01`，换句话说，普通人群中有 1% 的人患有糖尿病（免责声明：这些值只是假设，并非任何医学研究的结论）。

`P(Pos)`：是获得阳性测试结果的概率。

`P(Neg)`：是获得阴性测试结果的概率。

`P(Pos|D)`：是本身有糖尿病并且获得阳性测试结果的概率，值为 `0.9`，换句话说，该测试在 90% 的情况下是正确的。亦称为敏感性或真正例率。

`P(Neg|~D)`：是本身没有糖尿病并且获得阴性测试结果的概率，值也为 `0.9` ，因此在 90% 的情况下是正确的。亦称为特异性或真负例率。

贝叶斯公式如下所示：

<img src="images/bayes_formula.png" height="242" width="242">

* `P(A)`：A 独立发生的先验概率。在我们的示例中为 `P(D)`，该值已经提供给我们了 。

* `P(B)`：B 独立发生的先验概率。在我们的示例中为 `P(Pos)`。

* `P(A|B)`：在给定 B 的情况下 A 发生的后验概率，在我们的示例中为 `P(D|Pos)`，即**某人的测试结果为阳性时患有糖尿病的概率。这是我们要计算的值。**

* `P(B|A)`：在给定 A 的情况下 B 可能发生的概率。在我们的示例中为 `P(Pos|D)`，该值已经提供给我们了 。

将这些值代入贝叶斯定理公式中：

`P(D|Pos) = P(D) * P(Pos|D) / P(Pos)`

获得阳性测试结果 `P(Pos)` 的概率可以使用敏感性和特异性来计算，如下所示：

`P(Pos) = [P(D) * Sensitivity] + [P(~D) * (1-Specificity))]`

In [20]:
# P(D)
p_diabetes = 0.01

# P(~D)
p_no_diabetes = 0.99

# Sensitivity or P(Pos|D)
p_pos_diabetes = 0.9

# Specificity or P(Neg|~D)
p_neg_no_diabetes = 0.9

# P(Pos)
p_pos = p_diabetes * p_pos_diabetes + p_no_diabetes * p_neg_no_diabetes
print('The probability of getting a positive test result P(Pos) is: {}',format(p_pos))

The probability of getting a positive test result P(Pos) is: {} 0.9


** 我们可以利用所有这些信息计算后验概率，如下所示：**
​    
某人测试结果为阳性时患有糖尿病的概率为：

`P(D|Pos) = (P(D) * Sensitivity)) / P(Pos)`

某人测试结果为阳性时没有糖尿病的概率为：

`P(~D|Pos) = (P(~D) * (1-Specificity)) / P(Pos)`

后验概率的和将始终为 `1`。

In [ ]:
'''
Instructions:
Compute the probability of an individual having diabetes, given that, that individual got a positive test result.
In other words, compute P(D|Pos).

The formula is: P(D|Pos) = (P(D) * P(Pos|D) / P(Pos)
'''

In [21]:
'''
Solution
'''
# P(D|Pos)
p_diabetes_pos = p_diabetes * p_pos_diabetes / p_pos
print('Probability of an individual having diabetes, given that that individual got a positive test result is:\
',format(p_diabetes_pos)) 

Probability of an individual having diabetes, given that that individual got a positive test result is: 0.01


In [ ]:
'''
Instructions:
Compute the probability of an individual not having diabetes, given that, that individual got a positive test result.
In other words, compute P(~D|Pos).

The formula is: P(~D|Pos) = P(~D) * P(Pos|~D) / P(Pos)

Note that P(Pos|~D) can be computed as 1 - P(Neg|~D). 

Therefore:
P(Pos|~D) = p_pos_no_diabetes = 1 - 0.9 = 0.1
'''

In [23]:
'''
Solution
'''
# P(Pos|~D)
p_pos_no_diabetes = 0.1

# P(~D|Pos)
p_no_diabetes_pos = p_pos_no_diabetes * p_no_diabetes / p_pos
print ('Probability of an individual not having diabetes, given that that individual got a positive test result is:'\
,p_no_diabetes_pos)

Probability of an individual not having diabetes, given that that individual got a positive test result is: 0.11


恭喜！你从头实现了贝叶斯定理。你的分析表明即使某人的测试结果为阳性，他/她也有 8.3% 的概率实际上患有糖尿病，以及 91.67% 的概率没有糖尿病。当然前提是全球只有 1% 的人群患有糖尿病，这只是个假设。

** “朴素贝叶斯”中的“朴素”一词是什么意思？** 

朴素贝叶斯中的“朴素”一词实际上是指，算法在进行预测时使用的特征相互之间是独立的，朴素贝叶斯是贝叶斯定理的一种延伸，假设所有特征相互之间是独立的。

### 第 4.2 步：从头实现朴素贝叶斯 ###

现在我们将用它来考虑有多个特征的情况。

假设有两个政党的候选人，“Jill Stein”是绿党候选人，“Gary Johnson”是自由党的候选人，两位候选人在演讲中提到“自由”、“移民”和“环境”这些字眼的概率为：

* Jill Stein 提到“自由”的概率：0.1 ---------> `P(F|J)`
* Jill Stein 提到“移民”的概率：0.1 -----> `P(I|J)`
* Jill Stein 提到“环境”的概率：0.8 -----> `P(E|J)`


* Gary Johnson 提到“自由”的概率：0.7 -------> `P(F|G)`
* Gary Johnson 提到“移民”的概率：0.2 ---> `P(I|G)`
* Gary Johnson 提到“环境”的概率：0.1 ---> `P(E|G)`


假设 Jill Stein 发表演讲的概率 `P(J)` 是 `0.5`，Gary Johnson 也是 `P(G) = 0.5`。


了解这些信息后，如果我们要计算 Jill Stein 提到“自由”和“移民”的概率，该怎么做呢？这时候朴素贝叶斯定理就派上用场了，我们将考虑两个特征：“自由”和“移民”。

现在我们可以定义朴素贝叶斯定理的公式：

<img src="images/naivebayes.png" height="342" width="342">

在该公式中，`y` 是分类变量，即候选人的姓名，`x1` 到 `xn` 是特征向量，即单个单词。该定理假设每个特征向量或单词 (`xi`) 相互之间是独立的。

为了详细讲解该公式，我们需要计算以下后验概率：

* `P(J|F,I)`：Jill Stein 提到“自由”和“移民”的概率。

* `P(G|F,I)`：Gary Johnson 提到“自由”和“移民”的概率。

In [24]:
# P(J)
p_j = 0.5

# P(F/J)
p_j_f = 0.1

# P(I/J)
p_j_i = 0.1

p_j_text = p_j * p_j_f * p_j_i
print(p_j_text)

0.005000000000000001


In [26]:
# P(G)
p_g = 0.5

# P(F/G)
p_g_f = 0.7

# P(I/G)
p_g_i = 0.2

p_g_text = p_g * p_g_f * p_g_i
print(p_g_text)

0.06999999999999999


In [27]:
p_f_i = p_j_text + p_g_text
print('Probability of words freedom and immigration being said are: ', format(p_f_i))

Probability of words freedom and immigration being said are:  0.075


现在可以计算 `P(J|F,I)` 的概率，即 Jill Stein 提到“自由”和“移民”的概率，以及 `P(G|F,I)`，即 Gary Johnson 提到“自由”和“移民”的概率。

In [ ]:
'''
Instructions:
Compute P(J|F,I) using the formula P(J|F,I) = (P(J) * P(F|J) * P(I|J)) / P(F,I) and store it in a variable p_j_fi
'''

In [28]:
p_j_fi = p_j_text / p_f_i
print('The probability of Jill Stein saying the words Freedom and Immigration: ', format(p_j_fi))

The probability of Jill Stein saying the words Freedom and Immigration:  0.06666666666666668


In [ ]:
'''
Instructions:
Compute P(G|F,I) using the formula P(G|F,I) = (P(G) * P(F|G) * P(I|G)) / P(F,I) and store it in a variable p_g_fi
'''

In [30]:
p_g_fi = p_g_text / p_f_i
print('The probability of Gary Johnson saying the words Freedom and Immigration: ', format(p_g_fi))

The probability of Gary Johnson saying the words Freedom and Immigration:  0.9333333333333332


### 5 ：使用 scikit-learn 实现朴素贝叶斯 ###

In [ ]:
'''
Instructions:

We have loaded the training data into the variable 'training_data' and the testing data into the 
variable 'testing_data'.

Import the MultinomialNB classifier and fit the training data into the classifier using fit(). Name your classifier
'naive_bayes'. You will be training the classifier using 'training_data' and y_train' from our split earlier. 
'''

In [33]:
from sklearn.naive_bayes import MultinomialNB
naive_bayes = MultinomialNB()
naive_bayes.fit(training_data, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [34]:
predictions = naive_bayes.predict(testing_data)

### 第 6 步：评估模型 ###

In [36]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('Accuracy score: ', format(accuracy_score(predictions, y_test)))
print('Precision score: ', format(precision_score(predictions, y_test)))
print('Recall score: ', format(recall_score(predictions, y_test)))
print('F1 score: ', format(f1_score(predictions, y_test)))

Accuracy score:  0.9885139985642498
Precision score:  0.9405405405405406
Recall score:  0.9720670391061452
F1 score:  0.9560439560439562


### 第 7 步：总结 ###

和其他分类算法相比，朴素贝叶斯具有的一大主要优势是能够处理大量特征。在我们的示例中，有数千个不同的单词，每个单词都被当做一个特征。此外，即使存在不相关的特征也有很好的效果，不容易受到这种特征的影响。另一个主要优势是相对比较简单。朴素贝叶斯完全可以直接使用，很少需要调整参数，除非通常分布数据已知的情况需要调整。
它很少会过拟合数据。另一个重要优势是相对于它能处理的数据量来说，训练和预测速度很快。